In [47]:
from PIL import Image, ImageDraw
import os
import json

# 데이터셋 위치
dataset_path = "D:/1__Data/공모전 - 2DSS"

# 라벨별 색 지정
class_colors = {
    'sky': (135, 206, 235),           # Light Blue
    'static': (128, 128, 128),        # Gray
    'road': (169, 169, 169),          # Dark Gray
    'pole/structural object': (139, 69, 19), # Brown
    'vegetation': (34, 139, 34),      # Green
    'building': (255, 165, 0),        # Orange
    'sidewalk': (255, 255, 0),        # Yellow
    'road boundaries': (255, 0, 0),   # Red
    'pedestrian': (0, 0, 255),        # Blue
    'vehicle': (128, 0, 128),         # Purple
    'traffic light': (255, 69, 0),    # Red-Orange
    'crosswalks': (255, 255, 255),    # White
    'lane': (255, 20, 147),           # Pink
    'traffic sign': (0, 255, 255),    # Cyan
    'dynamic': (0, 0, 0)              # Black
}


def create_label_image(data, image_size):
    """다각형 데이터를 사용하여 라벨 이미지를 생성하는 함수"""
    image = Image.new("RGB", tuple(image_size), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    for annotation in data['Annotation']:
        class_name = annotation['class_name']
        color = class_colors.get(class_name, (0, 0, 0))  # 해당 클래스의 색상 가져오기, 기본은 검정색
        points = annotation['data'][0]  # 다각형의 점 좌표
        polygon = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
        draw.polygon(polygon, outline="black", fill=color)  # 외곽선과 채우기 색상 지정

    return image

def process_case(case_path):
    """각 케이스의 이미지와 라벨을 처리하는 함수"""
    image_path = os.path.join(case_path, 'images')
    label_path = os.path.join(case_path, 'labels')
    save_path = os.path.join(case_path, 'label_image')

    if not os.path.isdir(save_path):
        os.makedirs(save_path)

    label_files = os.listdir(label_path)

    for json_name in label_files:
        json_file_path = os.path.join(label_path, json_name)
        with open(json_file_path, 'r') as file:
            data = json.load(file)
            image_size = data['image_size']
            label_image = create_label_image(data, image_size)
            label_image.save(os.path.join(save_path, json_name.replace('.json', '.jpg')))

def main(dataset_path):
    """데이터셋 경로에서 모든 케이스를 처리하는 메인 함수"""
    for case in os.listdir(dataset_path):
        case_path = os.path.join(dataset_path, case)
        if os.path.isdir(case_path):  # validation / test / train
            process_case(case_path)

if __name__ == "__main__":
    main(dataset_path)
